In [1]:
!pip install nsepython

Defaulting to user installation because normal site-packages is not writeable


In [3]:
!pip install matplotlib

Defaulting to user installation because normal site-packages is not writeable


In [4]:
from nsepython import nse_optionchain_scrapper
from datetime import datetime
import numpy as np
from scipy.stats import norm
import pandas as pd
import matplotlib.pyplot as plt

In [50]:
def black_scholes(S, K, T, r, sigma, option_type='call'):
    if T <= 0 or sigma <= 0:
        return 0 
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type == "Call":
        price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    else:
        price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)

    return price

In [12]:
data = nse_optionchain_scrapper("NIFTY")

In [36]:
spot_price = float(data['records']['underlyingValue'])

In [66]:
records = []

for item in data['records']['data']:
    strike = item['strikePrice']
    
    if 'CE' in item:
        ce = item['CE']
        if ce['totalTradedVolume'] > 500:
            records.append({
                'type': 'call',
                'strike': strike,
                'ltp': ce['lastPrice'],
                'volume': ce['totalTradedVolume'],
                'expiry': ce['expiryDate'],
                'iv': ce['impliedVolatility'] / 100  # convert % to decimal
            })
    
    if 'PE' in item:
        pe = item['PE']
        if pe['totalTradedVolume'] > 500:
            records.append({
                'type': 'put',
                'strike': strike,
                'ltp': pe['lastPrice'],
                'volume': pe['totalTradedVolume'],
                'expiry': pe['expiryDate'],
                'iv': pe['impliedVolatility'] / 100
            })

df = pd.DataFrame(records)
df

,type,strike,ltp,volume,expiry,iv
0,put,21000,36.35,803,24-Dec-2025,0.1884
1,put,22000,16.40,1878,25-Sep-2025,0.1842
2,put,22000,55.05,1111,24-Dec-2025,0.1683
3,put,22250,0.85,11429,31-Jul-2025,0.3057
4,put,22300,0.80,6657,31-Jul-2025,0.2990
...,...,...,...,...,...,...
379,call,27400,6.75,1357,28-Aug-2025,0.1148
380,call,28000,14.75,1969,25-Sep-2025,0.1121
381,call,28000,118.00,3869,24-Dec-2025,0.0924
382,call,29000,47.10,1284,24-Dec-2025,0.0992


In [67]:
min_vol = 50
df = df[df['volume'] > min_vol].reset_index(drop=True)
df

,type,strike,ltp,volume,expiry,iv
0,put,21000,36.35,803,24-Dec-2025,0.1884
1,put,22000,16.40,1878,25-Sep-2025,0.1842
2,put,22000,55.05,1111,24-Dec-2025,0.1683
3,put,22250,0.85,11429,31-Jul-2025,0.3057
4,put,22300,0.80,6657,31-Jul-2025,0.2990
...,...,...,...,...,...,...
379,call,27400,6.75,1357,28-Aug-2025,0.1148
380,call,28000,14.75,1969,25-Sep-2025,0.1121
381,call,28000,118.00,3869,24-Dec-2025,0.0924
382,call,29000,47.10,1284,24-Dec-2025,0.0992


In [68]:
now = pd.Timestamp.now()
df['expiry_dt'] = pd.to_datetime(df['expiry'], format="%d-%b-%Y")
df['T'] = (df['expiry_dt'] - now).dt.total_seconds() / (365 * 24 * 60 * 60)
df['T'].apply(lambda x: f"{x:.7f}")
df = df.sort_values(by='T', ascending=False).reset_index(drop=True)
df

,type,strike,ltp,volume,expiry,iv,expiry_dt,T
0,put,21000,36.35,803,24-Dec-2025,0.1884,2025-12-24,0.420208
1,put,23000,100.80,2322,24-Dec-2025,0.1538,2025-12-24,0.420208
2,call,29000,47.10,1284,24-Dec-2025,0.0992,2025-12-24,0.420208
3,call,28000,118.00,3869,24-Dec-2025,0.0924,2025-12-24,0.420208
4,call,27000,312.60,3258,24-Dec-2025,0.0900,2025-12-24,0.420208
...,...,...,...,...,...,...,...,...
379,call,25100,131.00,5476726,24-Jul-2025,0.1174,2025-07-24,0.001030
380,put,25100,18.20,5827202,24-Jul-2025,0.1063,2025-07-24,0.001030
381,call,25850,0.65,93213,24-Jul-2025,0.2034,2025-07-24,0.001030
382,put,23750,0.75,32817,24-Jul-2025,0.4233,2025-07-24,0.001030


In [69]:
df = df[df['T'] > 0]
df

,type,strike,ltp,volume,expiry,iv,expiry_dt,T
0,put,21000,36.35,803,24-Dec-2025,0.1884,2025-12-24,0.420208
1,put,23000,100.80,2322,24-Dec-2025,0.1538,2025-12-24,0.420208
2,call,29000,47.10,1284,24-Dec-2025,0.0992,2025-12-24,0.420208
3,call,28000,118.00,3869,24-Dec-2025,0.0924,2025-12-24,0.420208
4,call,27000,312.60,3258,24-Dec-2025,0.0900,2025-12-24,0.420208
...,...,...,...,...,...,...,...,...
379,call,25100,131.00,5476726,24-Jul-2025,0.1174,2025-07-24,0.001030
380,put,25100,18.20,5827202,24-Jul-2025,0.1063,2025-07-24,0.001030
381,call,25850,0.65,93213,24-Jul-2025,0.2034,2025-07-24,0.001030
382,put,23750,0.75,32817,24-Jul-2025,0.4233,2025-07-24,0.001030


In [40]:
risk_free_rate = 0.07

In [70]:
df['bs_price'] = df.apply(
    lambda row: black_scholes(
        S=spot_price,
        K=row['strike'],
        T=row['T'],
        r=risk_free_rate,
        sigma=row['iv'],
        option_type=row['type']
    ),
    axis=1
)
df

,type,strike,ltp,volume,expiry,iv,expiry_dt,T,bs_price
0,put,21000,36.35,803,24-Dec-2025,0.1884,2025-12-24,0.420208,46.434171
1,put,23000,100.80,2322,24-Dec-2025,0.1538,2025-12-24,0.420208,129.217885
2,call,29000,47.10,1284,24-Dec-2025,0.0992,2025-12-24,0.420208,2985.772943
3,call,28000,118.00,3869,24-Dec-2025,0.0924,2025-12-24,0.420208,2061.983505
4,call,27000,312.60,3258,24-Dec-2025,0.0900,2025-12-24,0.420208,1237.230092
...,...,...,...,...,...,...,...,...,...
379,call,25100,131.00,5476726,24-Jul-2025,0.1174,2025-07-24,0.001030,6.409822
380,put,25100,18.20,5827202,24-Jul-2025,0.1063,2025-07-24,0.001030,4.592281
381,call,25850,0.65,93213,24-Jul-2025,0.2034,2025-07-24,0.001030,645.038770
382,put,23750,0.75,32817,24-Jul-2025,0.4233,2025-07-24,0.001030,0.000418


In [76]:
df['abs_error'] = np.abs(df['bs_price'] - df['ltp'])
df['pct_error'] = 100 * df['abs_error'] / df['ltp']
df['days_to_expiry'] = np.ceil((df['expiry_dt'] - datetime.now()).dt.total_seconds() / 86400).astype(int)
df

,type,strike,ltp,volume,expiry,iv,expiry_dt,T,bs_price,abs_error,pct_error,days_to_expiry
0,put,21000,36.35,803,24-Dec-2025,0.1884,2025-12-24,0.420208,46.434171,10.084171,27.741875,154
1,put,23000,100.80,2322,24-Dec-2025,0.1538,2025-12-24,0.420208,129.217885,28.417885,28.192346,154
2,call,29000,47.10,1284,24-Dec-2025,0.0992,2025-12-24,0.420208,2985.772943,2938.672943,6239.220686,154
3,call,28000,118.00,3869,24-Dec-2025,0.0924,2025-12-24,0.420208,2061.983505,1943.983505,1647.443648,154
4,call,27000,312.60,3258,24-Dec-2025,0.0900,2025-12-24,0.420208,1237.230092,924.630092,295.786978,154
...,...,...,...,...,...,...,...,...,...,...,...,...
379,call,25100,131.00,5476726,24-Jul-2025,0.1174,2025-07-24,0.001030,6.409822,124.590178,95.107006,1
380,put,25100,18.20,5827202,24-Jul-2025,0.1063,2025-07-24,0.001030,4.592281,13.607719,74.767685,1
381,call,25850,0.65,93213,24-Jul-2025,0.2034,2025-07-24,0.001030,645.038770,644.388770,99136.733920,1
382,put,23750,0.75,32817,24-Jul-2025,0.4233,2025-07-24,0.001030,0.000418,0.749582,99.944242,1


In [78]:
error_by_days = df.groupby('days_to_expiry')['pct_error'].agg(['count', 'mean', 'median', 'std']).reset_index()
print(error_by_days.sort_values('days_to_expiry'))

   days_to_expiry  count           mean     median            std
0               1    114  110971.048172  99.974719  204838.943976
1               8    117   33086.802752  34.719431   71955.056271
2              15     51    2700.114868   6.386731   12349.343637
3              22      7     198.621983   7.932077     446.088453
4              36     55    1505.646936  11.592031    4742.259601
5              64     28     900.227703  17.384918    3147.143734
6             154     12    1928.973250  29.061542    4402.339789
